In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.metrics import auc
import pandas as pd
import random
import seaborn as sns
import pickle as pkl

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

In [2]:
sp_dims = (12, 4) #(width, height)
v_max, v_min = 5, -1
xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
y_max, y_min = 6, -2

---

In [3]:
# paired_rec = 'rsc'
paired_rec = 'hpc'

# v1LGN_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1LGN_spks_CQuinn.pkl")
# v1LGN_psth = pd.read_hdf(r"U:\Papers\D&P Osc in Mouse Vis Sys\Data Analysis\Units\V1_LGN\V1_LGN_PSTH_Final.hdf5")
# v1LP_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1LP_spks_CQuinn.pkl")
# v1LP_psth = pd.read_hdf('u:/Papers/D&P Osc in Mouse Vis Sys/Data Analysis/Units/V1_LP/LP_PSTH_remLT_300sps_remGT-0o10ampWF_LP_GUI.hdf5')

if paired_rec == 'rsc':
    # v1other_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1RSC_spks_CQuinn.pkl")
    v1other_psth = pd.read_hdf('u:/Papers/D&P Osc in Mouse Vis Sys/Data Analysis/Units/V1_RSC/V1_RSC_2kmeans_PSTH_GUI_mapped_sliced.hdf5')
elif paired_rec == 'hpc':
    # v1other_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1HPC_spks_CQuinn.pkl")
    v1other_psth = pd.read_pickle(r"U:\Papers\D&P Osc in Mouse Vis Sys\New Submission\RESUBMISSION\dataFrames\V1HPC_psth.pkl")
# v1other_psth.head()

In [5]:
vr_resp_df = v1other_psth[v1other_psth.r_group_type == 'excited']

if paired_rec == 'rsc':
    vr_1situ = vr_resp_df[(vr_resp_df.condition == 'GroupWTpostG-1-G---GRSC')|(vr_resp_df.condition == 'GroupWTpostG-1-G---GV1')]
elif paired_rec == 'hpc':
    vr_1situ = vr_resp_df[vr_resp_df.stim == 'post']

In [ ]:
if paired_rec == 'rsc':
    group_A_df = vr_1situ[vr_1situ['location'] == 'V1']
    group_B_df = vr_1situ[vr_1situ['location'] == 'RSC']
elif paired_rec == 'hpc':
    group_A_df = vr_1situ[vr_1situ['region'] == 'v1']
    group_B_df = vr_1situ[vr_1situ['region'] == 'hippo']

A_array = np.array(group_A_df.pivot(index='cuid', columns='times', values='zscore'))
print(f'v1: {A_array.shape}') #this is [units x times] and the values are above

B_array = np.array(group_B_df.pivot(index='cuid', columns='times', values='zscore'))
print(f'hpc: {B_array.shape}') #this is [units x times] and the values are above

In [7]:
#--------------------------------------------------------------------------------
if paired_rec == 'rsc':
    num_cycles_required = 4
elif paired_rec == 'hpc':
    num_cycles_required = 3
#--------------------------------------------------------------------------------

durationA, indA, cuidA_ls, include_cuidA = [], [], [], []
for unit in range(A_array.shape[0]):
    yy = A_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        durationA.append(dur)
        indA.append(peak_ind)
        cuidA_ls.append(yy)
        include_cuidA.append(True)
    else:
        include_cuidA.append(False)

durationB, indB, cuidB_ls, include_cuidB = [], [], [], []
for unit in range(B_array.shape[0]):
    yy = B_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        durationB.append(dur)
        indB.append(peak_ind)
        cuidB_ls.append(yy)
        include_cuidB.append(True)
    else:
        include_cuidB.append(False)

indA_array = np.concatenate(indA, axis=0)
indB_array = np.concatenate(indB, axis=0)
cuidA_arr = np.array(cuidA_ls)
cuidB_arr = np.array(cuidB_ls)

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=(10,3))
f.tight_layout()
# plt.suptitle(f'{stim_choice} 3 cycles')

# Heatmap for A
sns.heatmap(cuidA_arr, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_ylabel(str(cuidA_arr.shape[0]))
ax1.set_title('V1')

#Heatmap for B
sns.heatmap(cuidB_arr, cmap = 'afmhot', xticklabels=20, yticklabels=False, vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_ylabel(str(cuidB_arr.shape[0]))
ax2.set_title(paired_rec)

plt.show()

In [10]:
#get the cuid for the above units somehow...

In [ ]:
print(f'{include_cuidA.count(True)} out of {len(include_cuidA)} V1 units')
print(f'{include_cuidB.count(True)} out of {len(include_cuidB)} {paired_rec} units')

In [ ]:
resA = [ele for ele in include_cuidA for i in range(group_A_df.times.nunique())]
resB = [ele for ele in include_cuidB for i in range(group_B_df.times.nunique())]

group_A_df['has_cycles'] = resA
group_B_df['has_cycles'] = resB

In [ ]:
#get the list of cuid for each brain region that has_cycles=True

V1_units_cycling = list(group_A_df[group_A_df.has_cycles == True]['cuid'].unique())
BR_units_cycling = list(group_B_df[group_B_df.has_cycles == True]['cuid'].unique())

all_units_cycling = {'V1':V1_units_cycling, 'Other':BR_units_cycling}

print(len(V1_units_cycling), len(BR_units_cycling))

In [ ]:
print(len(all_units_cycling['V1']), len(all_units_cycling['Other']))

In [145]:
# save all_units cycling list to a csv and send to C.Quinn

save_path = r'C:\Users\AChub_Lab\Desktop\V1HPC_cycling_units.pkl'

with open(save_path, 'wb') as f:
    pkl.dump(all_units_cycling, f)